<a href="https://colab.research.google.com/github/yn2a75/benkyokai/blob/main/pdf2text_ExtractCompanyName_RenameFle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#必要なライブラリをインストール
! pip install pdfminer.six
! pip install pandas
! pip install openpyxl
! pip install xlwt
! pip install xlsxwriter

### PDFファイルからテキストを出力する関数を作成

In [ ]:
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams
import os
from io import StringIO

#PDFからテキストを出力する関数を定義
def convert_pdf2txt(filename):
  pdf_file = os.path.join(pdffiles_folder, filename)
  fp = open(pdf_file, 'rb')
    
  outfp = StringIO()
        
  rmgr = PDFResourceManager()
  lprms = LAParams()
  device = TextConverter(rmgr, outfp, laparams=lprms)
  iprtr = PDFPageInterpreter(rmgr, device)
    
  for page in PDFPage.get_pages(fp):
    iprtr.process_page(page)

#テキスト内容を取得し、改行とスペースを削除
  text = outfp.getvalue()
  text=text.replace("\n", "")
  text=text.replace("   ", "\n　")
  text=text.replace(" ", "")

#テキスト内容を返す
  return text

In [ ]:
#GoogleDriveをマウント
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#PDFを保管しているフォルダのpathを指定

pdffiles_folder=r"/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/PDFfiles"

#テキストを格納する空のリストを作成
text_list=[]


filelist = os.listdir(pdffiles_folder)
filelist = [f for f in filelist if '.pdf' in f]
pdfpaths = [os.path.join(pdffiles_folder,f) for f in filelist if '.pdf' in f]

#リストにPDFのテキストを追加
for filename in filelist:
  text_list.append(convert_pdf2txt(filename))

In [ ]:
len(text_list)
len(filelist)
len(pdfpaths)
text_list

['2022年３月10日\n\u3000会\n\u3000社\n\u3000名\n\u3000\n\u3000株式会社シマノ代表者名\n\u3000\n\u3000代表取締役社長\n\u3000島野泰三(コード番号:7309東証第一部)\n\u3000問合せ先\n\u3000\n\u3000取締役総務管理部長兼経理部管掌\n\u3000\n\u3000金井琢磨(ＴＥＬ.072-223-3254)第115期定時株主総会における議案に関する補足説明について2022年３月30日開催予定の第115期定時株主総会（以下「本株主総会」という）における第２号議案（取締役４名選任の件）、第３号議案（監査役３名選任の件）に関し下記の事項を補足いたします。本株主総会において取締役及び監査役の選任議案が原案どおり承認可決され、さらに執行役員制度導入に伴い執行役員となる現取締役の退任及び本株主総会終了後開催の取締役会で予定されている決議の内容が反映された場合、当社の取締役及び監査役の構成は以下の表のとおりとなります。（１）取締役（８名、うち社外取締役３名）役職代表取締役会長兼CEO代表取締役社長代表取締役副社長津崎祥博代表取締役副社長チアチンセン常務取締役Shimano(Singapore)Pte.Ltd.社長（２）監査役（４名、うち社外監査役２名）役職野末佳奈子社外監査役橋本敏彦社外監査役以上氏名島野容三島野泰三豊嶋敬一條和生勝丸充啓榊原定征氏名平田義弘樽谷潔記社外取締役社外取締役社外取締役常勤監査役常勤監査役\n\u3000各位\n\u3000\x0c',
 '2022年10月期第1四半期決算説明資料トビラシステムズ株式会社（東証一部4441）\x0c1事業概要22022年10月期第1四半期決算概要INDEX目次3中期経営計画の進捗状況4参考資料2\x0c事業概要13\x0c企業理念と行動指針テクノロジーで社会の課題解決に挑戦する私たちの生活私たちの世界をよりよい未来につなぐトビラになる企業理念行動指針私たちは素晴らしい未来を想像し、失敗を恐れず変化を続け、常識を疑いあるべき形を追求します。社会的課題を解決する、人々の役に立つ製品を次々に生み出し、持続的かつ発展的に成長するため適切な利益を得ます。自分と大切な人が幸せな時間を送れる環境であり続けることが、私たちの成長や世界中の人々の生活

### テキストから会社名を抜き出す
ginzaとspacyを使う

In [ ]:
#ライブラリをインストール
!pip install -U ginza
!pip install -U ginza ja-ginza

In [ ]:
#spacyをインポートしてインスタンス化
import spacy
from spacy import displacy
nlp = spacy.load('ja_ginza')

OrderDictを使ってリストから、順序を維持したうえで重複を削除したリストを作成する
https://www.delftstack.com/ja/howto/python/remove-duplicates-from-list-python/

In [ ]:
#OrderDictをインポート
from collections import OrderedDict

### テキストから会社名を返す関数を作成

In [ ]:
#テキストから会社名を返す関数をつくる

def Text2CompanyName(text):
  doc=nlp(text)
  #空のリストを準備
  company_names_=[]
#固有表現ラベルがCompanyをリストに追加
  for ent in doc.ents: 
    if ent.label_ =='Company':
      #print(ent.text, ent.label_)
      company_names_.append(ent.text) 
#      company_names=list(set(company_names_))
      company_names=list(OrderedDict.fromkeys(company_names_))
      #StrCompanyName = "".join(company_names)
  if len(company_names_)==0:
    return ["なし"]
  else:
    return company_names

### 参考
listが空かどうか判定する方法
https://qiita.com/yonedaco/items/d0f65ca3dad2e085a51d

### 用意した関数を使い、テキストから会社名を抽出して新たなリストを作成する

In [ ]:
#companyの名前を入れる空のリストを準備
companyNames1=[]
for company in text_list:
    companyNames1.append(Text2CompanyName(company))

### 2次元リストの各行の0番目値を取得して新たにリストを作る  
https://ja.stackoverflow.com/questions/47331/python%E3%81%A7%E4%BA%8C%E6%AC%A1%E5%85%83%E9%85%8D%E5%88%97%E3%81%AE%E4%B8%AD%E3%81%AE%E5%90%84%E8%A6%81%E7%B4%A0%E3%81%AEn%E7%95%AA%E7%9B%AE%E3%81%A0%E3%81%91%E3%82%92%E5%8F%96%E3%82%8A%E5%87%BA%E3%81%97%E3%81%A6-%E8%A6%81%E7%B4%A0%E3%81%A8%E3%81%97%E3%81%A6%E4%B8%A6%E3%81%B9%E3%81%9F%E3%81%84

In [ ]:
#CompanyName1の各行の0番目の値をCompanyNames2に格納
companyNames2=list(map(lambda companyName1:companyName1[0],companyNames1))

### pandasでエクセル表のもととなる一覧表を作成
https://www.delftstack.com/ja/howto/python-pandas/pandas-create-dataframe-from-list/

In [ ]:
#pandasをインポート
import pandas as pd

In [ ]:
#データフレームを作成
df=pd.DataFrame(list(zip(pdfpaths,filelist,text_list,companyNames1,companyNames2)),columns=["path","filename","text","companynames","companyname"])

In [ ]:
df

,path,filename,text,companynames,companyname
0,/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/P...,140120220310503726.pdf,2022年３月10日\n 会\n 社\n 名\n \n 株式会社シマノ代表者名\n \n 代...,"[株式会社シマノ, 定時株主総会, Singapore)Pte.Ltd.]",株式会社シマノ
1,/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/P...,140120220310503389.pdf,2022年10月期第1四半期決算説明資料トビラシステムズ株式会社（東証一部4441） 1事業...,"[株式会社（東証一部4441, 会社概要, トビラシステムズ株式会社, TalkBook, ...",株式会社（東証一部4441
2,/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/P...,140120220310503662.pdf,各位\n \n 2022年３月10日会社名テ代表者名代表取締役社長ラ株式会真\n 船\n 達...,"[定時株主総会, HLBMeisei有限責任監査法人, HLBMeisei]",定時株主総会


In [ ]:
#データフレームをエクセル出力
output_excel_path=pdffiles_folder + "list.xlsx"
df.to_excel(output_excel_path , engine='xlsxwriter')

### Pandasでエクセルファイルに書き出そうとすると表示される「IllegalCharacterError」を回避
pandasの書き出しエンジンをxlsxwriterに変更する  
df.to_excel("test.xlsx", engine='xlsxwriter')  
https://karupoimou.hatenablog.com/entry/2019/09/29/235528

### PDFのファイル名を変更し、別フォルダに移動して保存する。
https://note.nkmk.me/python-shutil-move/

In [ ]:
#会社名を確認したエクセルをデータフレームに読み込む
df2 = pd.read_excel(output_excel_path)

In [ ]:
#移動先のフォルダを指定
#strtodir=pdffiles_folder +"\\" + "processed"
strtodir='/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/PDFfiles/processed'

In [ ]:
#新しいファイル名を含めた移動先のパスを指定
df2["topath"]=strtodir + "/" +df2["companyname"] + df2["filename"] +".pdf"

In [ ]:
#ファイルを移動するためのモジュールをインポート
import shutil
import os

In [ ]:
#if文でファイルが存在するかを確認した上で、ファイルを移動する
for path, topath in zip(df2["path"],df2["topath"]):
    if os.path.exists(path):
        new_path=shutil.move(path,topath)
    else:
        print("file in not exist")
#    print(path,topath)